In [1]:
from script.runtime import *

# await load('http://127.0.0.1:8188/')
await load()

# Nodes can only be imported after load()
from script.runtime.nodes import *

Nodes: 167


In [4]:
async with TaskManager():
    model, clip, vae = CheckpointLoaderSimple('v1-5-pruned-emaonly.ckpt')
    conditioning = CLIPTextEncode('beautiful scenery nature glass bottle landscape, , purple galaxy bottle,', clip)
    conditioning2 = CLIPTextEncode('text, watermark', clip)
    latent = EmptyLatentImage(512, 512, 1)
    latent = KSampler(model, 156680208700286, 20, 8, 'euler', 'normal', conditioning, conditioning2, latent, 1)
    image = VAEDecode(latent, vae)
    SaveImage(image, 'ComfyUI')

Queue remaining: 1


100%|██████████████████████████████████████████████████| 20/20
Queue remaining: 0


In [5]:
await interrupt_current()

In [6]:
await clear_queue()

In [7]:
await interrupt_all()

In [8]:
model, clip, vae = CheckpointLoaderSimple('v1-5-pruned-emaonly.ckpt')
conditioning = CLIPTextEncode('beautiful scenery nature glass bottle landscape, , purple galaxy bottle,', clip)
conditioning2 = CLIPTextEncode('text, watermark', clip)
latent = EmptyLatentImage(512, 512, 1)
latent = KSampler(model, 156680208700286, 20, 8, 'euler', 'normal', conditioning, conditioning2, latent, 1)
image = VAEDecode(latent, vae)
save = SaveImage(image, 'ComfyUI')

print(await queue(save))

{'prompt_id': 'dc03e7fa-daba-4843-a059-07b1e04cf9c4', 'number': 12, 'node_errors': {}}
Queue remaining: 1
Queue remaining: 0
